# Модуль 3.2 — Типы агентов (Functions vs ReAct)

**Цель:** понять разницу между агентами, которые вызывают инструменты «по функции», и агентами, которые рассуждают пошагово.

**Что сделаем:**
- установим библиотеки
- настроим ключ и base URL
- построим двух агентов: Function‑style и ReAct
- сравним поведение и рекомендации

In [ ]:
%pip -q install -U \
  langchain \
  langchain-openai \
  langchain-community \
  pydantic==2.12.3 \
  python-dotenv \
  tiktoken \
  requests==2.32.4

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Общая идея

- **Function‑style агент** старается вызывать инструменты строго и точно.
- **ReAct‑агент** рассуждает пошагово и выбирает инструмент по ходу.

Мы используем один и тот же инструмент и сравним поведение.

In [ ]:
from pydantic import BaseModel, Field
from langchain.tools import BaseTool

class SearchProductsInput(BaseModel):
    query: str = Field(description="Поисковый запрос на русском языке")
    max_results: int = Field(5, ge=1, le=5)

class ProductSearchTool(BaseTool):
    name = "product_search"
    description = "Поиск товаров в каталоге"
    args_schema = SearchProductsInput

    def _run(self, query: str, max_results: int = 5) -> str:
        catalog = [
            "Беспроводные наушники AirSound",
            "Умная лампа BrightHome",
            "Фитнес‑браслет FitGo",
            "Электронная книга ReadLite",
        ]
        results = [item for item in catalog if query.lower() in item.lower()]
        return "\n".join(results[:max_results]) or "Ничего не найдено"

tool = ProductSearchTool()

## Function‑style агент

Этот агент предпочитает вызывать инструменты «по схеме», без лишних рассуждений.
Он хорош, когда важна точность параметров и предсказуемость.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

base_url = os.environ.get("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.2,
    max_tokens=256,
    base_url=base_url,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты ассистент, который строго вызывает инструменты при необходимости."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

functions_agent = create_openai_functions_agent(llm, [tool], prompt)
functions_executor = AgentExecutor(agent=functions_agent, tools=[tool], verbose=False)

result = functions_executor.invoke({"input": "Найди лампу в каталоге"})
print(result["output"])

## ReAct‑агент

ReAct строит рассуждение «мысль → действие → наблюдение». Он гибче, но может быть менее предсказуем.

In [ ]:
from langchain.agents import create_react_agent

react_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Ты ассистент, который может рассуждать и использовать инструменты."),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

react_agent = create_react_agent(llm, [tool], react_prompt)
react_executor = AgentExecutor(agent=react_agent, tools=[tool], verbose=False)

react_result = react_executor.invoke({"input": "Найди лампу в каталоге"})
print(react_result["output"])

## Сравнение и рекомендации

**Function‑style**:
- точнее вызывает инструменты
- лучше для строгих API и бизнес‑логики

**ReAct**:
- гибче при сложных задачах
- может «рассуждать» лишнее и чаще ошибаться

**Практика:**
- Для продакшена чаще выбирают Function‑style.
- ReAct полезен для исследовательских задач и сложных пайплайнов.